Bayesian Optimization with 3 layer MLP for COPD detection

In [ ]:
# import lightgbm as lgb
import numpy as np
import pandas as pd
# from hyperopt import fmin, hp, tpe, Trials
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df=pd.read_csv('/kaggle/input/cardiovascular-disease-dataset/cardio_train.csv',sep=";")

In [ ]:
df.drop(['id'],axis=1,inplace=True)

In [ ]:
#Why this could be useful?
#Replicate the results for sensors
#compliance of MOX2-5 sensors

In [ ]:
df.head()

In [ ]:
df['BMI']=df['weight']/(df['height']/100)**2
X=df.drop(['cardio'],axis=1)
y=df['cardio']
X_train, X_test, y_train, y_test =train_test_split(X,y)

In [ ]:

space = {
    'num_leaves': hp.quniform('num_leaves', 10, 200, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.5)),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
}


def objective(params):
    clf =  MLPClassifier(hidden_layer_sizes=(50,), activation='relu', max_iter=500, random_state=42)
    score = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc').mean()
    return -score


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials,
            verbose=1)

print("Meilleurs hyperparamètres:", best)

In [ ]:
best['hidden_layer_sizes']=int(best['hidden_layer_sizes'])
best['activation']=int(best['activation'])
best['max_iter']=int(best['max_iter'])
clf=MLPClassifier(**best)
clf.fit(X_train,y_train)


In [ ]:
y_pred=clf.predict(X_test)
accuracy_score(y_pred,y_test)